# RISC Zero Soundness Analysis - Unique Decoding Regime
(Originally written by the team at Ulvetanna; this version is an in-process translation of that work to the unique decoding regime.)

The goal is to concretely compute the overall error estimate $\varepsilon_{\text{IOP}}$ from the [[RISC Zero](https://www.risczero.com/proof-system-in-detail.pdf)] spec. 

Let's begin by defining some global constants, which depend only on the proof system. These are all taken directly from the spec, with the exception of `h` and `L`, which we obtained by inspecting [the open-source code](https://github.com/risc0/risc0).

In [103]:
import math

k = 2  # negative log of the rate.
rho = 1 / (1 << k)  # rate.
h = 20  # log of the trace length. this value refers to the default segment size. 
H = 1 << h  # domain size.
D = H / rho  # domain size, after low-degree extension.
e = 4  # field extension degree.
p = (1 << 32) - (1 << 27) + 1  # field characteristic: Baby Bear prime.
F = math.pow(p, e)  # extension field size.
C = 16 + 222 + 36  # number of control, data, and accumulation columns, respectively. retrieved from `circuit.get_taps()`, etc.
L = C + 4  # number of polynomials which ultimately appear in the FRI batch. we need to add 4 extra columns, for the "segment polynomials".
s = 150  # number of FRI queries.

TODO - update this section for UDR 
The parameter `m` below is the "Johnson parameter"; see [[Haböck](https://eprint.iacr.org/2022/1216.pdf)]. This one is subtle, since it doesn't appear directly in the protocol at all; rather, it only appears in the soundness estimates. We have chosen it to here _optimize_ the number of bits of provable security achieved. We comment on this further below.

In [104]:
m = 16 # TODO update for UDR... do m and m+ both disappear entirely? 
# i think there will still be some notion of rho+...
alpha = 1 - (1 - rho) / 2 # TODO factcheck whether the "1 - " is correct here
theta = 1 - alpha

### Provable Soundness

The below error terms reflect the _proven_ / known soundness, and derive from Theorem 1.2 of [[Proximity Gaps](https://eprint.iacr.org/2020/654.pdf)]. The below expressions are based on [[Haböck](https://eprint.iacr.org/2022/1216.pdf)] and [[RISC Zero](https://www.risczero.com/proof-system-in-detail.pdf)], which are in agreement.

The assertion that e_FRI_constant = (L + 1) * e_proximity_gap is based on the analysis [here](https://hackmd.io/-GVDIdGOT-yiQgFjZA-Ayw).

In [105]:
e_proximity_gap = D / F
e_FRI_constant = (L + 1) * e_proximity_gap 
e_FRI_queries = math.pow(1 - theta, s)
e_FRI = e_FRI_constant + e_FRI_queries

TODO -- revise this note for unique decoding radius

There's a further subtlety. In order for the proof of [[Haböck](https://eprint.iacr.org/2022/1216.pdf), Theorem 8] to go through, we need to apply FRI with proximity parameter $\theta < 1 - \sqrt{\rho^+}$ (note the +!). In particular, we can't choose `m` _too_ large, or else $\theta := 1 - \sqrt{\rho} \cdot \left( 1 + \frac{1}{2 \cdot m} \right)$ will become larger than $\frac{1 - \rho}{2}$, and the key step in that theorem will fail. Specifically, the relevant _configuration's_ distance from Reed–Solomon code $\mathsf{RS}_{k^+}(K, D)$ over the function field $K := F(Z)$ will exceed the Johnson bound of that code. For our parameters, this doesn't happen until `m = 131073`, so we have plenty of room.

Separately, when we calculate $L^+$, we need to use a _different_ `m` than the one up above. Specifically, we need to use $m^+$, say, where $m^+$ is chosen sufficiently large that $\theta \leq 1 - \sqrt{\rho^+} \cdot \left( 1 + \frac{1}{2 \cdot m^+} \right)$ holds. This fact is not explicit in [[Haböck](https://eprint.iacr.org/2022/1216.pdf), Theorem 8], and appears to have been an oversight.

Let's calculate a minimal suitable $m^+$:

In [106]:
rho_plus = (H + 2) / D  # modified rate of "extension code" in function field; controls the agreement we actually need
# todo adjust this assertion for unique decoding radius
assert theta < 1 - math.sqrt(rho_plus)  # make sure `m` isn't too large.
m_plus = math.ceil(1 / (2 * (alpha / math.sqrt(rho_plus) - 1)))
# todo adjust this assertion for unique decoding radius
assert theta <= 1 - math.sqrt(rho_plus) * (1 + 1 / (2 * m_plus))
m_plus

3

In [107]:
L_plus = 1 # Since we restrict our analysis to the unique decoding radius, we set the list-size to be 1.

e_ALI = L_plus * C / F
e_DEEP = L_plus * (4 * (H + 1) + (H - 1)) / (F - H - D)
e_PLONK = e * 5 * H / F  # 5 comes from n_{σ_{mem}} == 5.
e_PLOOKUP = e * 15 * H / F  # n_{σ_{bytes}} == 15.

e_IOP = e_FRI + e_ALI + e_DEEP + e_PLONK + e_PLOOKUP
math.log(e_IOP, 2)

-99.28880159528796

Thus it seems that 150 queries is sufficient to reach ~100 bits of proven security, overall. 

The individual values are collected in the table below:

In [108]:
import numpy as np
import pandas as pd

def show_table():
    err_terms = pd.DataFrame(
        [
            ["e_FRI_constant", e_FRI_constant],
            ["e_FRI_queries", e_FRI_queries],
            ["e_ALI", e_ALI],
            ["e_DEEP", e_DEEP],
            ["e_PLONK", e_PLONK],
            ["e_PLOOKUP", e_PLOOKUP],
        ],
        columns=["Term", "P[err]"]
    )
    err_terms["log_2(P[err])"] = np.log2(err_terms["P[err]"])
    err_terms = err_terms.drop(columns="P[err]")
    return err_terms
show_table()

,Term,log_2(P[err])
0,e_FRI_constant,-99.692664
1,e_FRI_queries,-101.710786
2,e_ALI,-119.718753
3,e_DEEP,-107.494854
4,e_PLONK,-105.494857
5,e_PLOOKUP,-103.909895
